## free-marketing-watch
Search social media for mentions of brands and collect the comments/tweets/etc.
Count mentions of each and perform sentiment analysis on the strings.

In [1]:
import praw
import pandas as pd
from secrets import *
from pathlib import Path
from brands import fashion
import re

In [2]:
reddit = praw.Reddit(client_id=client_id,
               client_secret=client_secret,
               user_agent=user_agent)

Now to get the comments data, put it in a dataframe, and clean the data to get what we want.

In [3]:
def create_comments_df(subreddit_):
    """Returns a pandas df with the information about comments from this year.

    Inputs
    -----
    str: subreddit to be searched.
    Return
    ------
    Pandas dataframe with all the data from the praw object.m
    """
    subreddit = reddit.subreddit(subreddit_)
    submission_list = subreddit.search(
        'WAYWT',sort='new', time_filter="year", limit=1000
    )  # generator of submissions in the subreddit
    comment_list = []
    for submission in submission_list:
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list():
            comment_list.append(comment)

    df = pd.DataFrame([vars(comment) for comment in comment_list])
    df2 = df.loc[:,['link_id','id','score','body']]
    df2['Subreddit'] = subreddit_
    return df2


In [4]:
def brand_check(df,brandlist):
    """Checks comment body against a list of brands to see if it mentions any.
       Adds what brand was found if any in the brands column.
       
       Inputs
       ------
       Dataframe you will search over and a list of brands in a separate file. 
       Return
       ------
       Dataframe with column indicating what brand was found in the values.
       """

    df2 = df.body.str.extractall(pat=brandlist, flags=re.VERBOSE)
    df2.dropna(axis=0,how='all',inplace=True)
    return df2

### This takes a long time, probably around 30 minutes per 100,000 comments.

In [ ]:
df1 = create_comments_df('femalefashionadvice')
df1

Run to export the df to csv. Careful about overwriting. Use the mode = 'a' line to add to an existing file.


In [ ]:
p = Path.cwd() / 'data' / 'waywtdf.csv'
#df1.to_csv(path_or_buf = p)
df1.to_csv(path_or_buf = p, mode = 'a', header=False)

In [5]:
p = Path.cwd() / 'data' / 'waywtdf.csv'
df = pd.read_csv(p)
df

,Unnamed: 0,link_id,id,score,body,Subreddit
0,0,t3_klxncy,ghc7s6s,12,Really like the crooklyn hat and Santa coat gu...,malefashionadvice
1,1,t3_klxncy,ghbh5m6,4,Thank you to everyone who participated [in yes...,malefashionadvice
2,2,t3_klxncy,ghczbnz,11,Finally a real and unbiased member of the publ...,malefashionadvice
3,3,t3_klxncy,ghdw1as,3,"I believe that's Frank Rossitano. Yeah, probab...",malefashionadvice
4,4,t3_klxncy,ghcr9kf,2,Add funny to that list. What a freaking all star.,malefashionadvice
...,...,...,...,...,...,...
33802,13370,t3_eif7u5,fcrvhlu,3,thank you! i think i got it at hot topic many ...,femalefashionadvice
33803,13371,t3_eif7u5,fct07im,1,"Hahaha I did not! Class of '09 in NC, but say ...",femalefashionadvice
33804,13372,t3_eif7u5,fct28pu,5,"I will! Your aesthetic reminds me of her, too,...",femalefashionadvice
33805,13373,t3_eif7u5,fct406p,2,a solid aspiration! i wonder if she still carr...,femalefashionadvice


If you prefer pickling over csv, use these cells for IO

In [ ]:
p = Path.cwd() / 'data' / 'commentdf.pkl'
df.to_pickle(path = p)

In [ ]:
df = pd.read_pickle(filepath_or_buffer=p)

If you read from CSV it created an extra column so this cell drops it.

In [6]:
df = df.drop(columns =['Unnamed: 0'])

In [7]:
df2 = brand_check(df,fashion)

### Drop any columns that didn't find a match.

In [8]:
pd.set_option("display.max_rows",50, "display.max_columns", None)
df2.dropna(axis=1,how='all',inplace=True)
df2

uniqlo  Gap     HM Levis Carhartt BrooksBrothers Apple Patagonia  \
      match                                                                    
20    38       NaN  NaN    NaN   NaN      NaN            NaN   NaN       NaN   
64    1119     NaN  NaN    NaN   NaN      NaN            NaN   NaN       NaN   
66    114      NaN  NaN    NaN   NaN      NaN            NaN   NaN       NaN   
68    310      NaN  NaN    NaN   NaN      NaN            NaN   NaN       NaN   
      479      NaN  NaN    NaN   NaN      NaN            NaN   NaN       NaN   
...            ...  ...    ...   ...      ...            ...   ...       ...   
33773 222      NaN  NaN    NaN   NaN      NaN            NaN   NaN       NaN   
33774 1795     NaN  NaN    NaN   NaN      NaN            NaN   NaN       NaN   
33777 700      NaN  NaN   H&M    NaN      NaN            NaN   NaN       NaN   
33778 4415     NaN  NaN    NaN   NaN      NaN            NaN   NaN       NaN   
33801 25       NaN  NaN    NaN   NaN      NaN            NaN   NaN       NaN   

            Everlane JCrew Zara Target Vans BananaRepbulic OldNavy  \
      match                                                          
20    38         NaN   NaN  NaN    NaN  NaN            NaN     NaN   
64    1119       NaN   NaN  NaN    NaN  NaN            NaN     NaN   
66    114        NaN   NaN  NaN    NaN  NaN            NaN     NaN   
68    310        NaN   NaN  NaN    NaN  NaN            NaN     NaN   
      479        NaN   NaN  NaN    NaN  NaN            NaN     NaN   
...              ...   ...  ...    ...  ...            ...     ...   
33773 222        NaN   NaN  NaN    NaN  NaN            NaN     NaN   
33774 1795       NaN   NaN  NaN    NaN  NaN            NaN     NaN   
33777 700        NaN   NaN  NaN    NaN  NaN            NaN     NaN   
33778 4415       NaN   NaN  NaN    NaN  NaN             BR     NaN   
33801 25         NaN   NaN  NaN    NaN  NaN            NaN     NaN   

            SaintLaurent Prada CanadaGoose CommonProjects AllendEdmonds Nike  \
      match                                                                    
20    38             NaN   NaN         NaN            NaN           NaN  NaN   
64    1119           NaN   NaN         NaN            NaN           NaN  NaN   
66    114            NaN   NaN         NaN            NaN           NaN  NaN   
68    310            NaN   NaN         NaN            NaN           NaN  NaN   
      479            NaN   NaN         NaN            NaN           NaN  NaN   
...                  ...   ...         ...            ...           ...  ...   
33773 222            NaN   NaN         NaN            NaN           NaN  NaN   
33774 1795           NaN   NaN         NaN            NaN           NaN  NaN   
33777 700            NaN   NaN         NaN            NaN           NaN  NaN   
33778 4415           NaN   NaN         NaN            NaN           NaN  NaN   
33801 25             NaN   NaN         NaN            NaN           NaN  NaN   

            Adidas Abercrombie  Amazon CalvinKlein NorthFace RalphLauren  \
      match                                                                
20    38       NaN         NaN     NaN         NaN       NaN         NaN   
64    1119     NaN         NaN     NaN         NaN       NaN         NaN   
66    114      NaN         NaN     NaN         NaN       NaN         NaN   
68    310      NaN         NaN  Amazon         NaN       NaN         NaN   
      479      NaN         NaN  Amazon         NaN       NaN         NaN   
...            ...         ...     ...         ...       ...         ...   
33773 222      NaN         NaN     NaN         NaN       NaN         NaN   
33774 1795     NaN         NaN     NaN         NaN       NaN         NaN   
33777 700      NaN         NaN     NaN         NaN       NaN         NaN   
33778 4415     NaN         NaN     NaN         NaN       NaN         NaN   
33801 25       NaN         NaN     NaN         NaN       NaN         NaN   

            Lacoste TommyHilfi

In [9]:
overall_counts = df2.count().sort_values(ascending=False)

### Splitting up the male and female subreddits to get demographic data.

In [10]:
filt = (df['Subreddit'] == 'malefashionadvice')
mfa_df = df[filt]
filt2 = (df['Subreddit'] == 'femalefashionadvice')
ffa_df = df[filt2]

In [11]:
mfa_df = brand_check(mfa_df,fashion)
ffa_df = brand_check(ffa_df,fashion)

In [12]:
mfa_df.dropna(axis=1,how='all',inplace=True)
ffa_df.dropna(axis=1,how='all',inplace=True)

In [13]:
mfa_counts = mfa_df.count().sort_values(ascending=False)
ffa_counts = ffa_df.count().sort_values(ascending=False)

In [14]:
pd.set_option("display.max_rows",117)
counts_df = pd.concat([overall_counts,mfa_counts,ffa_counts],axis='columns',sort=False)

## Final dataframe with the number of mentions for each brand split into overall, r/malefashionadvice, and r/femalefashionadvice. Ready for data visualization.

In [20]:
counts_df.columns = ['Total','r/malefashionadvice','r/femalefashionadvice']
counts_df

,Total,r/malefashionadvice,r/femalefashionadvice
uniqlo,1077,801.0,276.0
Thrifted,881,434.0,447.0
Levis,425,327.0,98.0
JCrew,282,241.0,41.0
HM,249,107.0,142.0
BananaRepbulic,206,148.0,58.0
Target,192,77.0,115.0
Converse,175,126.0,49.0
Nike,174,140.0,34.0
Madewell,164,22.0,142.0
